In [93]:
import pandas as pd

In [94]:
data = pd.read_csv('modeling_data_0801.csv')

C:\Users\laptop\AppData\Local\Temp\ipykernel_59240\3775562660.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('modeling_data_0801.csv')


In [95]:
data.shape

(505491, 11)

In [96]:
data.head()

,user_pseudo_id,session_id,event_name,event_timestamp,session_duration,customer_type,original_customer_type,device_category,traffic_source,traffic_source_medium,time_to_conversion
0,+JCOGYuIB0UiMuCAfoa19Zx4/kyAS1mL/oiJpA8Efc4=.1...,1753451127,page_view,1753451127470651,250944587,NaN,NaN,desktop,app.jotform.com,referral,242274347
1,+JCOGYuIB0UiMuCAfoa19Zx4/kyAS1mL/oiJpA8Efc4=.1...,1753451127,first_visit,1753451127470651,250944587,NaN,NaN,desktop,app.jotform.com,referral,242274347
2,+JCOGYuIB0UiMuCAfoa19Zx4/kyAS1mL/oiJpA8Efc4=.1...,1753451127,session_start,1753451127470651,250944587,NaN,NaN,desktop,app.jotform.com,referral,242274347
3,+JCOGYuIB0UiMuCAfoa19Zx4/kyAS1mL/oiJpA8Efc4=.1...,1753451127,page_view,1753451132664814,250944587,NaN,NaN,desktop,app.jotform.com,referral,242274347
4,+JCOGYuIB0UiMuCAfoa19Zx4/kyAS1mL/oiJpA8Efc4=.1...,1753451127,view_item_list,1753451138041042,250944587,NaN,NaN,desktop,app.jotform.com,referral,242274347


In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505491 entries, 0 to 505490
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   user_pseudo_id          505491 non-null  object
 1   session_id              505491 non-null  int64 
 2   event_name              505491 non-null  object
 3   event_timestamp         505491 non-null  int64 
 4   session_duration        505491 non-null  int64 
 5   customer_type           361788 non-null  object
 6   original_customer_type  80108 non-null   object
 7   device_category         505491 non-null  object
 8   traffic_source          504824 non-null  object
 9   traffic_source_medium   504707 non-null  object
 10  time_to_conversion      505491 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 42.4+ MB


In [98]:
data = data.drop(columns=['event_name', 'event_timestamp', 'original_customer_type'])
data.shape

(505491, 8)

In [99]:
# drop duplicate rows
data = data.drop_duplicates()
data.shape

(7594, 8)

In [100]:
data['time_to_conversion'] = data['time_to_conversion'].apply(
    lambda x: x/1000000/60) # convert from microseconds to minutes

### Calculate average time to conversion by source

In [101]:
data['traffic_source'].value_counts()

traffic_source
google                                    3544
(direct)                                  2508
bing                                      1109
hs_email                                   127
CJ                                          70
Data Not Available                          42
yahoo                                       20
statics.teams.cdn.office.net                17
magento                                     16
mrcnet                                      15
excel.officeapps.live.com                   10
hs_automation                                9
duckduckgo                                   9
ebiz-myus-com.go.akamai-access.com           7
utm_source=magento                           6
sites.google.com                             4
PRODUCTAD                                    4
mail.google.com                              4
chatgpt.com                                  4
assistant.order.co                           3
kb.iflyworld.com                             

In [106]:
# filter only the traffic sources that have more than 10 occurrences
traffic_source = data[data['traffic_source'].isin(data['traffic_source'].value_counts()[data['traffic_source'].value_counts() > 10].index)]

traffic_source = traffic_source[['traffic_source', 'time_to_conversion']].groupby(['traffic_source']).mean().reset_index()
traffic_source.sort_values(by='time_to_conversion', ascending=False, inplace=True)
traffic_source.reset_index(drop=True, inplace=True)

In [107]:
traffic_source

,traffic_source,time_to_conversion
0,google,179.448373
1,(direct),75.345824
2,statics.teams.cdn.office.net,59.473321
3,hs_email,45.020417
4,bing,32.270053
5,CJ,26.233918
6,yahoo,21.377813
7,magento,19.262691
8,Data Not Available,17.040746
9,mrcnet,15.401458


In [104]:
data['traffic_source_medium'].value_counts()

traffic_source_medium
cpc                                                       3420
(none)                                                    2508
organic                                                   1262
email                                                      152
referral                                                   114
Data Not Available                                          42
Skimlinks                                                   24
Ebates Performance Marketing, Inc. dba Rakuten Rewards      14
Sovrn Inc                                                   12
Neuintel, LLC                                               11
Wildfire Systems                                             6
utm_medium=email                                             6
Demand.io                                                    2
banner                                                       2
EHSTODAY                                                     2
SH                               

In [110]:
source_medium = data[data['traffic_source_medium'].isin(data['traffic_source_medium'].value_counts()[data['traffic_source_medium'].value_counts() > 10].index)]

source_medium = source_medium[['traffic_source_medium', 'time_to_conversion']].groupby(['traffic_source_medium']).mean().reset_index()
source_medium.sort_values(by='time_to_conversion', ascending=False, inplace=True)
source_medium.reset_index(drop=True, inplace=True)

In [111]:
source_medium

,traffic_source_medium,time_to_conversion
0,cpc,172.232783
1,(none),75.345824
2,organic,65.983474
3,"Neuintel, LLC",57.378418
4,email,43.230481
5,Sovrn Inc,41.300238
6,referral,28.461716
7,Data Not Available,17.040746
8,Skimlinks,13.783710
9,"Ebates Performance Marketing, Inc. dba Rakuten...",10.790612


### Calculate average time to conversion by user type

In [91]:
user_type = data[['customer_type', 'time_to_conversion']].groupby(['customer_type']).mean().reset_index()
user_type.sort_values(by='time_to_conversion', ascending=False, inplace=True)
user_type.reset_index(drop=True, inplace=True)

In [92]:
user_type

,customer_type,time_to_conversion
0,notLoggedIn,55.731448
1,web,49.901481
2,corporate,33.781160


In [114]:
data['customer_type'].value_counts()

customer_type
notLoggedIn    1979
corporate      1919
web            1787
Name: count, dtype: int64